## Ce que fait ce code : Préparation de la base

* 1. Charger les bases de données du zip sur le repo github
* 2. Merger et append les bases de données
* 2.1 Micro exploration, Etudier l'attrition
* 2.2 Merge puis append, Etudier l'attrition
* 2.3 Export de la base data_base_enrichie_non_clean

1. Telecharger les fichiers dta sur serveur local SSPcloud

Nos données sont dans un fichier zip sur GitHub. On crée une fonction qui va chercher les données à l'intérieur du fichier zip; cette fonction charge aussi ces données sur le serveur local

Il faut en premier lieu : 

* cloner le git, et se positionner sur le dossier avec les donnees

In [3]:
#Code terminal : cloner le dossier avec les données github, utiles à l'extraction
!git clone https://github.com/willyafalin/monocentric-model-colombia-2021

Cloning into 'monocentric-model-colombia-2021'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 94 (delta 43), reused 62 (delta 24), pack-reused 0
Receiving objects: 100% (94/94), 19.24 MiB | 7.78 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [1]:
#Change de directory
!cd /home/onyxia/work/monocentric-model-colombia-2021


In [2]:
#On telecharge les commandes et libraries utiles à l'import des fichiers zip depuis le github
import requests
from io import BytesIO
from zipfile import ZipFile
import os


def download_and_extract_zip_from_github(owner, repo, zip_file_path, output_dir):
    # # URL du contenu brut GitHub pour le fichier zip
    raw_url = f'https://github.com/{owner}/{repo}/raw/main/{zip_file_path}'

    # Faire une demande pour télécharger le fichier zip
    response = requests.get(raw_url)

    if response.status_code == 200:
        # Extraire le contenu du fichier zip
        with ZipFile(BytesIO(response.content)) as zip_ref:
            # Extraire tous les fichiers dans le répertoire de sortie spécifié (local)
            zip_ref.extractall(output_dir)
            
            print(f"Files extracted to '{output_dir}'")
    else:
        print(f"Failed to download zip file. Status code: {response.status_code}")

# pour nos données
github_owner = 'willyafalin'
github_repo = 'monocentric-model-colombia-2021'
zip_file_path = 'data/Diciembre.dta.zip'
output_directory = '/home/onyxia/work/extracted_files' #ici, on met les donnees dans le serveur du SSP lab, pour eviter de faire des commits directs sur le clone github: en effet, si on ajoute toutes nos donnees unziped sur le dossier a commit, on va creer des erreurs
                                                       #contrepartie etant qu'il faut re-appeler ce dossier constamment, mais pas grave
download_and_extract_zip_from_github(github_owner, github_repo, zip_file_path, output_directory)

Files extracted to '/home/onyxia/work/extracted_files'


2. Creer un identifiant de merge merging_id et on ne garde que les ménages

Comme expliqué dans le markdown 0_main.ipynb, la DANE de Colombie collecte leurs données d'enquête sur les ménages en les stockant selon 3 différentes couches territoriales (capitale (area), périurbain (cabecerra), rural (rural)). On peut donc enrichir, pour une seule couche territoriale donnée, la base initiale des dépenses de logement avec celles contenant des informations sur d'autres variables sociocatégorielles (caractéristiques individuelles, données d'emploi ...). 
    - On verra dans la phase 2.a. d'exploration qu'enrichir les données de consommation sur le logement (ci-après 'data de logement') avec la base sur les caracteristiques individuelles suffit pour notre analyse. 

On va ainsi merger, pour chacune des 3 couches territoriales, les données de logement avec les variables sociocatégorielles d'intérêt. Ensuite, on va append ces 3 bases de données enrichies pour nous donner une base complète pour les répondants en decembre 2021. 

    2.a. Mini exploration avant de merger: on vérifie qu'il n'y a pas d'attrition intra-territoriale et après merge

Il faut trouver un idenfiant convenable pour éviter les mauvaises jointures (on veut matcher à 100% les bases de logement et caracteristiques individuelles d'une seule même couche territoriale). On commence par explorer les bases d'une seule couche territoriale (area) et on voit si les patterns dans les bases area s'appliquent pour celles des cabeceras et rural.

Premièrement, on détermine comment chaque individu (chaque ligne a priori) qui popule la base est identifié (peut être une seule variable ID, un couple, un tuple, peut être faut-il concatener plusieurs variables et construire une variable identifiante à cet effet). Cet ID devra permettre d'identifier un seul individu de façon unique, sans qu'il n'y ait de duplicatas. Cet identifieur sera aussi notre identifiant pour la merge entre les données de logement et caractéristiques individuelles, merging_id. 

Ensuite, on vérifie que toutes autres bases d'une seule même strate territoriale ont le même nombre total d'observations uniques, à partir de cet identifiant unique pour un seul ménage. 

Enfin, on s'assure qu'il n'y a pas eu d'attrition après appariement. 

        Exploration, Couche territoriale : area

In [3]:
import pandas as pd 

import warnings 
# on filtre les uniwarnings de ce block, pour eviter de les montrer dans le notebook 0_main // il concerne l'encodage utf-8 des dta
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UnicodeWarning)

# import des bases area

# importer la base dta principale // couche territoriale : area // base : consommation logement

    area_housing = pd.read_stata("/home/onyxia/work/extracted_files/DTA/╡rea - Vivienda y Hogares.DTA") 

# importer la 1ere base dta d'enrichissement // couche territoriale : area // base : caracteristiques individuelles

    area_carac = pd.read_stata("/home/onyxia/work/extracted_files/DTA/╡rea - Características generales (Personas).DTA") 


On verra plus bas qu'on aura juste besoin des bases "_carac" pour enrichir les bases "area_housing", "cabecera_housing", "rural_housing" 

In [4]:
# on garde les labels stockees dans une base stata dta ?

area_carac_label = pd.read_stata("/home/onyxia/work/extracted_files/DTA/╡rea - Características generales (Personas).DTA", iterator=True)
area_carac_label.variable_labels() #les labels ont le meme nom que les variables. donc inutile d'utiliser les labels. les labels dans la base dta produite par la DANE ne donnent pas un intitule de la variable
# et on continue l'analyse avec area_carac et non area_carac_label, pour eviter d'obtenir des messages d'erreurs a cause de la lecture en dta

{'DIRECTORIO': 'DIRECTORIO',
 'SECUENCIA_P': 'SECUENCIA_P',
 'ORDEN': 'ORDEN',
 'HOGAR': '',
 'REGIS': '',
 'P6016': 'P6016',
 'P6020': 'P6020',
 'P6030S1': 'P6030S1',
 'P6030S3': 'P6030S3',
 'P6040': 'P6040',
 'P6050': 'P6050',
 'P6083': 'P6083',
 'P6083S1': 'P6083S1',
 'P6081': 'P6081',
 'P6081S1': 'P6081S1',
 'P6080': 'P6080',
 'P6080S1': 'P6080S1',
 'P6070': 'P6070',
 'P6071': 'P6071',
 'P6071S1': 'P6071S1',
 'P3147S1': 'P3147S1',
 'P3147S2': 'P3147S2',
 'P3147S3': 'P3147S3',
 'P3147S4': 'P3147S4',
 'P3147S5': 'P3147S5',
 'P3147S6': 'P3147S6',
 'P3147S7': 'P3147S7',
 'P3147S8': 'P3147S8',
 'P3147S9': 'P3147S9',
 'P3147S10': 'P3147S10',
 'P3147S10A1': 'P3147S10A1',
 'P3147S11': 'P3147S11',
 'P3246': 'P3246',
 'P6090': 'P6090',
 'P6140': 'P6140',
 'P6150': 'P6150',
 'P6100': 'P6100',
 'P6110': 'P6110',
 'P6120': 'P6120',
 'P6125': 'P6125',
 'P6160': 'P6160',
 'P6170': 'P6170',
 'P6175': 'P6175',
 'P6210': 'P6210',
 'P6210S1': 'P6210S1',
 'P6220': 'P6220',
 'P6269': 'P6269',
 'AREA': 

In [5]:
area_housing_label = pd.read_stata("/home/onyxia/work/extracted_files/DTA/╡rea - Vivienda y Hogares.DTA", iterator=True)
area_housing_label.variable_labels()
#parel pour les labels de housing 

{'DIRECTORIO': 'DIRECTORIO',
 'SECUENCIA_P': 'SECUENCIA_P',
 'P5000': 'P5000',
 'P5010': 'P5010',
 'P5020': 'P5020',
 'P5030': 'P5030',
 'P5040': 'P5040',
 'P5050': 'P5050',
 'P5070': 'P5070',
 'P5080': 'P5080',
 'P5090': 'P5090',
 'P5090S1': 'P5090S1',
 'P5100': 'P5100',
 'P5110': 'P5110',
 'P5130': 'P5130',
 'P5140': 'P5140',
 'P5210S1': 'P5210S1',
 'P5210S2': 'P5210S2',
 'P5210S3': 'P5210S3',
 'P5210S4': 'P5210S4',
 'P5210S5': 'P5210S5',
 'P5210S6': 'P5210S6',
 'P5210S7': 'P5210S7',
 'P5210S8': 'P5210S8',
 'P5210S9': 'P5210S9',
 'P5210S10': 'P5210S10',
 'P5210S11': 'P5210S11',
 'P5210S14': 'P5210S14',
 'P5210S15': 'P5210S15',
 'P5210S16': 'P5210S16',
 'P5210S17': 'P5210S17',
 'P5210S18': 'P5210S18',
 'P5210S19': 'P5210S19',
 'P5210S20': 'P5210S20',
 'P5210S21': 'P5210S21',
 'P5210S22': 'P5210S22',
 'P5210S24': 'P5210S24',
 'P5220': 'P5220',
 'P5220S1': 'P5220S1',
 'P6008': 'P6008',
 'P6007': 'P6007',
 'P6007S1': 'P6007S1',
 'HOGAR': '',
 'P4000': 'P4000',
 'P4010': 'P4010',
 'P4020'

Du coup, on doit regarder le dictionnaire de variable, pour connaitre l'intitule precis d'une variable donnee : 
- [donnees caracteristiques individuelles (esp)](https://microdatos.dane.gov.co/index.php/catalog/701/data-dictionary/F2?file_name=%C3%81rea%20-%20Caracter%C3%ADsticas%20generales%20(Personas))
- [donnees logement (esp)](https://microdatos.dane.gov.co/index.php/catalog/701/data-dictionary/F1?file_name=%C3%81rea%20-%20Vivienda%20y%20Hogares)

A noter que ces dictionnaires de variables pour la couche area sont les memes pour les 2 autres couches territoriales (cabecera et rural) 

(les memes liens susmentionnés, traduits en anglais) : 
- [donnees caracteristiques individuelles (eng)](https://microdatos-dane-gov-co.translate.goog/index.php/catalog/701/data-dictionary/F2?file_name=%C3%81rea+-+Caracter%C3%ADsticas+generales+(Personas)&_x_tr_sl=auto&_x_tr_tl=en&_x_tr_hl=de&_x_tr_pto=wapp)
- [donnees logement (eng)](https://microdatos-dane-gov-co.translate.goog/index.php/catalog/701/data-dictionary/F1?file_name=%C3%81rea+-+Vivienda+y+Hogares&_x_tr_sl=auto&_x_tr_tl=en&_x_tr_hl=de&_x_tr_pto=wapp)

In [6]:
#visualisateur de la base

pd.set_option('display.max_columns', None) #on veut voir toutes les colonnes de la base
#pd.set_option('display.max_rows', None) #fait crasher si on fait avec les lignes. on va plutot en prendre quelques uns
pd.set_option('display.min_rows',10) #juste quelques lignes suffisent

area_carac #visualisateur, pour voir à quoi ressemble la base grosso modo
area_housing

,DIRECTORIO,SECUENCIA_P,P5000,P5010,P5020,P5030,P5040,P5050,P5070,P5080,P5090,P5090S1,P5100,P5110,P5130,P5140,P5210S1,P5210S2,P5210S3,P5210S4,P5210S5,P5210S6,P5210S7,P5210S8,P5210S9,P5210S10,P5210S11,P5210S14,P5210S15,P5210S16,P5210S17,P5210S18,P5210S19,P5210S20,P5210S21,P5210S22,P5210S24,P5220,P5220S1,P6008,P6007,P6007S1,HOGAR,P4000,P4010,P4020,P4030S1,P4030S1A1,P4030S2,P4030S3,P4030S4,P4030S4A1,P4030S5,P4040,REGIS,AREA,MES,DPTO,fex_c_2011
0,6037206.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,,NaN,NaN,NaN,800000.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,NaN,1.0,2.0,1.0,4.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,01,05,12,05,1296.092057
1,6037207.0,1.0,6.0,4.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,,NaN,NaN,NaN,750000.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,4.0,7.0,2.0,NaN,1.0,2.0,1.0,4.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,01,05,12,05,1451.291178
2,6037209.0,1.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,,NaN,NaN,NaN,700000.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,NaN,1.0,2.0,1.0,4.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,01,05,12,05,1193.103490
3,6037210.0,1.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,,NaN,NaN,NaN,750000.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,3.0,2.0,NaN,1.0,2.0,1.0,4.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,01,05,12,05,1155.402197
4,6037211.0,1.0,4.0,3.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,,NaN,NaN,NaN,740000.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,3.0,2.0,NaN,1.0,2.0,1.0,4.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,01,05,12,05,1094.942950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8486,6062991.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,,NaN,NaN,NaN,500000.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,4.0,2.0,NaN,1.0,2.0,1.0,4.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,01,11,12,11,5904.077240
8487,6062992.0,1.0,3.0,2.0,2.0,1.0,1.0,1.0,1.0,3.0,3.0,,NaN,NaN,NaN,300000.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,3.0,2.0,NaN,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0,01,23,12,23,257.674387
8488,6062993.0,1.0,3.0,2.0,2.0,1.0,1.0,1.0,1.0,3.0,4.0,,NaN,NaN,400000.0,NaN,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,4.0,2.0,NaN,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0,01,23,12,23,208.388368
8489,6062994.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,,NaN,NaN,350000.0,NaN,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,3.0,2.0,NaN,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,01,23,12,23,266.783327


On voit qu'on a des dimensions différentes au sein d'une seule couche territoriale, entre les caracteristiques individuelles (n = 26390 obs) et les donnees de logement (n = 8491 obs). On continue d'explorer rapidement pour trouver le bon ID pour coupler ces deux bases.
- chercher les variables totalement populées
- voir au sein de ces variables, lesquelles ont des duplicatas

In [7]:
#trouver les variables totalement populées dans les deux bases, voir où y a des valeurs manquantes

area_carac.describe().T #on transpose pour voir toutes les variables 
area_housing.describe().T


,count,mean,std,min,25%,50%,75%,max
DIRECTORIO,8491.0,6.048616e+06,8.018407e+03,6.037206e+06,6.041710e+06,6.047906e+06,6.055564e+06,6.062995e+06
SECUENCIA_P,8491.0,1.016724e+00,1.615682e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e+00
P5000,8491.0,3.350724e+00,1.099713e+00,1.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,1.100000e+01
P5010,8491.0,1.989518e+00,8.798267e-01,1.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,7.000000e+00
P5020,8491.0,1.027559e+00,2.639626e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.000000e+00
P5030,8481.0,1.036906e+00,1.885422e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
P5040,8491.0,1.016488e+00,2.464570e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e+00
P5050,8491.0,1.048286e+00,6.083636e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+01
P5070,8491.0,1.230479e+00,8.276688e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.000000e+00
P5080,8342.0,3.094102e+00,4.331777e-01,1.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,7.000000e+00


In [8]:
# on demande a avoir la liste des variables qui populent totalement la base // les variables potentielles pour creer identifiant de merge

non_missing_variables_area_carac = area_carac.columns[area_carac.notnull().all()]
non_missing_variables_area_housing = area_housing.columns[area_housing.notnull().all()]

# et on extrait une liste de variables communes aux deux listes 
non_missing_variables_area = list(set(non_missing_variables_area_carac) & set(non_missing_variables_area_housing)) #permet de ne pas tenir compte de l'ordre dans les deux listes // sinon merge 1:1 avec zip
non_missing_variables_area

['HOGAR',
 'MES',
 'DPTO',
 'REGIS',
 'AREA',
 'fex_c_2011',
 'SECUENCIA_P',
 'DIRECTORIO']

In [9]:
# on regarde le nombre d'occurrences des variables qui identifient uniquement un individu au sein d'un seul même ménage

# d'apres les dictionnaires de variables : 
# DIRECTORIO : identifiant de l'habitation // En Colombie, plusieurs ménages peuvent vivre sous un même type d'habitation, d'où la distinction 
# SECUENCIA_P : ordre du ménage au sein du DIRECTORIO // même raison qu'au dessus. mais cette information est redondante avec HOGAR
# HOGAR : numéro du ménage (foyer, famille)
# ORDEN : ordre du répondant // plusieurs répondants possible au sein d'un seul ménage DIRECTORIO-HOGAR

# donc a priori, on a seulement besoin de DIRECTORIO,HOGAR,ORDEN pour creer un idenfiant unique
# on peut deja s'apercevoir que, si area_carac est plus populee, c'est parce qu'elle incorpore des ORDEN supplementaires (donc la base de logement doit surement retenir 1 seul repondant par menage)
        # en effet, ORDEN est une variable qui n'apparait pas dans la base "_housing"

unique_area_carac = area_carac[['DIRECTORIO','SECUENCIA_P','HOGAR', 'ORDEN']].nunique()
unique_area_housing = area_housing[['DIRECTORIO','SECUENCIA_P','HOGAR']].nunique()

print(unique_area_carac) # on peut avoir jusqu'a 20 differents repondants dans un seul meme menage par exemple
print("//")
print(unique_area_housing)


DIRECTORIO     8383
SECUENCIA_P       5
HOGAR             5
ORDEN            20
dtype: int64
//
DIRECTORIO     8383
SECUENCIA_P       5
HOGAR             5
dtype: int64


In [10]:
area_carac['merge_id'] = area_carac['DIRECTORIO'].astype(str) + area_carac['HOGAR'].astype(str)
area_housing['merge_id'] = area_housing['DIRECTORIO'].astype(str) + area_housing['HOGAR'].astype(str)

area_carac[['DIRECTORIO','SECUENCIA_P','HOGAR', 'ORDEN','merge_id']].nunique(),area_housing[['DIRECTORIO','SECUENCIA_P','HOGAR','merge_id']].nunique()

(DIRECTORIO     8383
 SECUENCIA_P       5
 HOGAR             5
 ORDEN            20
 merge_id       8491
 dtype: int64,
 DIRECTORIO     8383
 SECUENCIA_P       5
 HOGAR             5
 merge_id       8491
 dtype: int64)

In [26]:
si tu veux ajouter des trucs dans le main 

from IPython.display import display, Markdown
display(Markdown("##ok")), display(

SyntaxError: invalid syntax (3448214948.py, line 1)

On obtient bien un identifiant unique dans la base area_housing (qui a 8491 entrées au total). Quant à la base area_carac, on voit que l'id de merge proposee pourrait convenir, comme le nombre d'uniques merge_id dans cette base correspond à celui dans la base area_housing 

Traitons maintenant les duplicatas directorio-hogar (plusieurs repondants orden au sein d'un couple directorio-hogar dans "_carac")

Les deux seules variables qui matchent parfaitement le nombre de variables dans area_housing: P3246 et P6050. Deux facons de verifier ensuite si ce sont de bonnes variables pour eviter des duplicatas ou de l'attrition lors de la merge puis append: 
- si ces deux variables permettent d'isoler les duplicatas, il semble alors que c'est le chef de famille qui repond a la question P3240 // et que c'est lui aussi qui est pris comme individu representatif dans les bases "_housing" (en annexe, on verra que toutes les autres bases travail, chomage, loisir, etc. ; sont aussi populees à 8491 obs)
- avec les poids d'inclusionfex_c_2011 si on garde la meme representativite (simplement si on garde le meme nombre d'uniques observations dans fex_c_2011)// voir si les merges avec les autres couches (cabecera, rural) marchent aussi sans attrition

  
Remarque : on voit que dans la base, c'est le chef du menage qui paie les consommations de logement. Donc il reste logique de prendre ses caracteristiques individuelles pour la suite de l'analyse, mais il faut garder l'information de la composition du menage (qu'on integrera avec les unites de consommation)

In [11]:

#chef de menage 
a = area_carac['P6050'].value_counts() # on tabule la frequence de la variable P6050
# unite de consommations sont importantes pour l'effet taille / composition du menage, à ajouter dans les traitements (2_data_cleaning)
# on voit que P6050==1 est le chef du menage (voir dictionnaire variable) et est populé avec 8491 observations

#P3246 : question "vous considerez-vous pauvre?" 
b = area_carac['P3246'].count()

#poids de sous echantillonage
c = area_carac['fex_c_2011'].nunique()
d = area_housing['fex_c_2011'].nunique()
#il faut qu'a la fin, les uniques fex_c_2011 matchent = new merge id, qu'on ajoute aux hogar machin

print(a,"//",b,"//",c,"//",d)

P6050
3.0    8923
1.0    8491
2.0    4311
5.0    2464
4.0    1857
9.0     291
6.0      45
7.0       5
8.0       3
Name: count, dtype: int64 // 8491 // 2927 // 2927


In [12]:
# traitement des multiples repondants, des duplicatas pour un couple directorio-hogar

area_carac_test = area_carac[area_carac['P6050'] == 1] #on garde les chef de menage, et on voit si P3246 est totalement populé

#attrition?
a= area_carac_test['P6050'].value_counts() #garde que les chefs de menage ==1
b= area_carac_test[['merge_id']].nunique() #on a maintenant des uniques directorio-hogar
print(a,"//",b, "//",c)
c= area_carac_test['fex_c_2011'].nunique() #on garde le meme nombre d'uniques fex_c_2011
area_carac_test.describe().T #P3246 totalement popule


P6050
1.0    8491
Name: count, dtype: int64 // merge_id    8491
dtype: int64 // 2927


,count,mean,std,min,25%,50%,75%,max
DIRECTORIO,8491.0,6.048616e+06,8018.406511,6.037206e+06,6.041710e+06,6.047906e+06,6.055564e+06,6.062995e+06
SECUENCIA_P,8491.0,1.016724e+00,0.161568,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e+00
ORDEN,8491.0,1.000000e+00,0.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
HOGAR,8491.0,1.016724e+00,0.161568,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e+00
P6016,8491.0,1.084089e+00,0.360598,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.000000e+00
P6020,8491.0,1.475091e+00,0.499409,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00
P6030S1,8327.0,6.620151e+00,3.435335,1.000000e+00,4.000000e+00,7.000000e+00,1.000000e+01,1.200000e+01
P6030S3,8327.0,1.969914e+03,16.405360,1.920000e+03,1.958000e+03,1.970000e+03,1.983000e+03,2.004000e+03
P6040,8491.0,5.113991e+01,16.430737,1.700000e+01,3.800000e+01,5.100000e+01,6.300000e+01,1.010000e+02
P6050,8491.0,1.000000e+00,0.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [13]:
# merge carac - housing

# on verifie qu'il y a toutes les observations depuis la base "_carac" 
# et apres avoir verifie qu'il n'y a pas de perte de donnees apres merge, on garde la base en jointure full_join
    # parce qu'on doit traiter les unites de consommations (donc on doit garder les caracteristiques des autres repondants
    # on n'aura pas besoin de grouper des variables (car on gardera les caracteristiques sociocategorielles de l'individu chef de famille
    # et aussi parce que les depenses logement et autres informations (travail, loisir, autres bases) se basent sur l'individu representatif // on n'a rien a grouper de toute maniere (les autres repondants sont NaN)

area_carac_housing = pd.merge(area_carac_test, area_housing, on='merge_id', how='inner', suffixes=('', '_y')) #jointure // on specifie les suffixes pour pouvoir garder une seule plus tard dans les analyses

a = len(area_carac_housing['merge_id'].value_counts())==area_carac_housing[['merge_id']].nunique() #meme nombre d'uniques et de count

print(a)

area_carac_housing.describe().T # on voit bien dans describe que les variables communes ont les memes caracteristiques
# il n'y a donc pas d'attrition


merge_id    True
dtype: bool


,count,mean,std,min,25%,50%,75%,max
DIRECTORIO,8491.0,6.048616e+06,8018.406511,6.037206e+06,6.041710e+06,6.047906e+06,6.055564e+06,6.062995e+06
SECUENCIA_P,8491.0,1.016724e+00,0.161568,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e+00
ORDEN,8491.0,1.000000e+00,0.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
HOGAR,8491.0,1.016724e+00,0.161568,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e+00
P6016,8491.0,1.084089e+00,0.360598,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.000000e+00
...,...,...,...,...,...,...,...,...
P4030S4,8491.0,1.004475e+00,0.066752,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
P4030S4A1,8453.0,2.872116e+00,0.660454,1.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,9.000000e+00
P4030S5,8491.0,1.004593e+00,0.067621,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
P4040,8452.0,1.143043e+00,0.350137,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00


Unité de consommation, traitement

La Colombie etant un pays de l'OCDE, on utilise le methode de la racine carrée pour tenir compte de la taille du ménage pendant l'étude

Sources : 

[OECD (2020) Compare your income methodology](https://www.oecd.org/statistics/Compare-your-income-methodology.pdf)

[OECD (n.d) Adjusting household incomes: equivalence scales](https://www.oecd.org/els/soc/OECD-Note-EquivalenceScales.pdf)


In [14]:
# aparté : traitement des unites de consommation 
#area_carac_test = area_carac_test.drop('taille_menage', axis=1)

import numpy as np

area_carac_new = area_carac #on repart de la base non traitée

area_carac_new['taille_menage'] = area_carac_new.groupby('merge_id')['DIRECTORIO'].transform('count') #on compte le nombre d'individus au sein d'un menage unique 

area_carac_new = area_carac_new.sort_values(by='DIRECTORIO') #ordonner la df

area_carac_new['uc'] = np.sqrt(area_carac_new['taille_menage']) #on applique la regle de la racine carree

area_carac_new


,DIRECTORIO,SECUENCIA_P,ORDEN,HOGAR,REGIS,P6016,P6020,P6030S1,P6030S3,P6040,P6050,P6083,P6083S1,P6081,P6081S1,P6080,P6080S1,P6070,P6071,P6071S1,P3147S1,P3147S2,P3147S3,P3147S4,P3147S5,P3147S6,P3147S7,P3147S8,P3147S9,P3147S10,P3147S10A1,P3147S11,P3246,P6090,P6140,P6150,P6100,P6110,P6120,P6125,P6160,P6170,P6175,P6210,P6210S1,P6220,P6269,AREA,ESC,MES,DPTO,fex_c_2011,merge_id,taille_menage,uc
0,6037206.0,1.0,1.0,1.0,10,1.0,1.0,1.0,1963.0,58.0,1.0,3.0,NaN,3.0,NaN,6.0,,3.0,1.0,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,,NaN,1.0,1.0,NaN,NaN,1.0,1.0,36341.0,1.0,1.0,2.0,NaN,5.0,11.0,2.0,NaN,05,11.0,12,05,1296.092057,6037206.01.0,2,1.414214
1,6037206.0,1.0,2.0,1.0,10,2.0,2.0,5.0,1964.0,57.0,2.0,3.0,NaN,3.0,NaN,6.0,,3.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0,1.0,1.0,NaN,1.0,NaN,,NaN,NaN,1.0,NaN,NaN,1.0,5.0,NaN,1.0,1.0,2.0,NaN,5.0,11.0,2.0,NaN,05,11.0,12,05,1296.092057,6037206.01.0,2,1.414214
6,6037207.0,1.0,5.0,1.0,10,5.0,1.0,10.0,1997.0,24.0,5.0,2.0,NaN,2.0,NaN,6.0,,2.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0,NaN,,NaN,NaN,2.0,1.0,48.0,NaN,NaN,NaN,2.0,1.0,2.0,NaN,5.0,11.0,2.0,NaN,05,11.0,12,05,1451.291178,6037207.01.0,7,2.645751
7,6037207.0,1.0,6.0,1.0,10,5.0,1.0,11.0,2019.0,2.0,4.0,1.0,4.0,1.0,5.0,6.0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,1.0,NaN,NaN,1.0,5.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05,NaN,12,05,1451.291178,6037207.01.0,7,2.645751
2,6037207.0,1.0,1.0,1.0,10,1.0,1.0,1.0,1979.0,42.0,1.0,2.0,NaN,2.0,NaN,6.0,,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,,NaN,2.0,1.0,NaN,NaN,1.0,1.0,36341.0,2.0,1.0,2.0,NaN,5.0,11.0,2.0,NaN,05,11.0,12,05,1451.291178,6037207.01.0,7,2.645751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26386,6062995.0,1.0,3.0,1.0,10,3.0,1.0,7.0,1980.0,41.0,3.0,1.0,2.0,1.0,1.0,6.0,,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,,NaN,NaN,1.0,NaN,NaN,1.0,1.0,48000.0,2.0,1.0,2.0,NaN,5.0,11.0,2.0,NaN,23,11.0,12,23,167.238369,6062995.01.0,6,2.449490
26384,6062995.0,1.0,1.0,1.0,10,1.0,1.0,1.0,1961.0,61.0,1.0,2.0,NaN,3.0,NaN,6.0,,3.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,,NaN,1.0,1.0,NaN,NaN,3.0,NaN,NaN,2.0,1.0,2.0,NaN,3.0,5.0,NaN,NaN,23,5.0,12,23,167.238369,6062995.01.0,6,2.449490
26385,6062995.0,1.0,2.0,1.0,10,2.0,2.0,4.0,1965.0,56.0,2.0,2.0,NaN,3.0,NaN,6.0,,3.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,,NaN,NaN,1.0,NaN,NaN,3.0,NaN,NaN,2.0,1.0,2.0,NaN,3.0,5.0,NaN,NaN,23,5.0,12,23,167.238369,6062995.01.0,6,2.449490
26387,6062995.0,1.0,4.0,1.0,10,4.0,1.0,7.0,2005.0,16.0,4.0,2.0,NaN,2.0,NaN,6.0,,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,,NaN,NaN,1.0,NaN,NaN,3.0,NaN,NaN,2.0,1.0,1.0,1.0,4.0,7.0,NaN,NaN,23,7.0,12,23,167.238369,6062995.01.0,6,2.449490


On recapitule : 
pour chaque couche territoriale,
- Creer le couple unique merge_id
- on prend la base de caracteristiques individuelles et on cree la variable unite de consommation (traitement composition menage)
- on applique à cette meme base un traitement pour supprimer les multiples repondants
- on merge cette base avec la base de logement
- on verifie bien que le nombre d'uniques merge_id correspond au nombre de count de merge_id (première verif attrition)

Ensuite, on append les 3 bases enrichies pour former notre base complete (les 3 couches territoriales avec les donnees individuelles et de logement)

Enfin, on vérifie que notre base enrichie soit représentative (deuxième verif attrition)


       2.b. Merge et append, Couches territoriale : cabecera et rural (et area)

In [15]:
# pour finir sur toutes les merges à faire, on fait une fonction 

def process_data(housing_data, carac_data, label):
    # donner du contexte 
    print( f"Nombre obs total carac {label}")
    ok = carac_data['DIRECTORIO'].count()
    print(ok)

    print( f"Nombre obs total housing {label}")
    ok2 = housing_data['DIRECTORIO'].count()
    print(ok2)
    
    # creer merge_id pour area_carac et area_housing
    carac_data['merge_id'] = carac_data['DIRECTORIO'].astype(str) + carac_data['HOGAR'].astype(str)
    housing_data['merge_id'] = housing_data['DIRECTORIO'].astype(str) + housing_data['HOGAR'].astype(str)

##### Traitement Composition ménage : compter le nombre d'individus au sein d'un menage unique
    carac_data['taille_menage'] = carac_data.groupby('merge_id')['DIRECTORIO'].transform('count')

    # appliquer la règle de la racine carrée
    carac_data['uc'] = np.sqrt(carac_data['taille_menage'])

    # garder uniquement les chefs de ménage)
    carac_data = carac_data[carac_data['P6050'] == 1]

    # jointure interne entre area_carac_test et area_housing
    data_merge = pd.merge(carac_data, housing_data, on='merge_id', how='inner', suffixes=('', '_y'))

    # construire le nom de la variable résultante avec le label
    result_variable_name = f"{label}_carac_housing"

##### Première vérification : verifier si le nombre d'identifiants merge_id uniques est égal au décompte
    if len(data_merge['merge_id'].value_counts()) == data_merge['merge_id'].nunique():
        print(f"Première vérif d'attrion : Pas d'attrition. Voici des stat. desc. de la base enrichie {label}_carac_housing:")
    else:
        print("Attrition")

    #ajouter une variable de couche_territoriale pour construire la variable territory plus tard
    data_merge['couche_territoriale'] = f"{label}"
    
    # Renommer la variable résultante avec le nom construit
    globals()[result_variable_name] = data_merge

    return data_merge.describe().T
    


In [16]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UnicodeWarning)

#cabecera
    c_housing = pd.read_stata("/home/onyxia/work/extracted_files/DTA/Cabecera - Vivienda y Hogares.DTA") 
    c_carac = pd.read_stata("/home/onyxia/work/extracted_files/DTA/Cabecera - Características generales (Personas).DTA") 

#rural 
    r_housing = pd.read_stata("/home/onyxia/work/extracted_files/DTA/Resto - Vivienda y Hogares.DTA") 
    r_carac = pd.read_stata("/home/onyxia/work/extracted_files/DTA/Resto - Caracteristicas generales (Personas).DTA") 


In [17]:
process_data(c_housing,c_carac, 'cabecera') #base enrichie cabecera

Nombre obs total carac cabecera
50664
Nombre obs total housing cabecera
16273
Première vérif d'attrion : Pas d'attrition. Voici des stat. desc. de la base enrichie cabecera_carac_housing:


,count,mean,std,min,25%,50%,75%,max
DIRECTORIO,16273.0,6.049328e+06,7775.831713,6.036443e+06,6.042073e+06,6.049210e+06,6.056148e+06,6.062995e+06
SECUENCIA_P,16273.0,1.013274e+00,0.142221,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e+00
ORDEN,16273.0,1.000000e+00,0.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
HOGAR,16273.0,1.013274e+00,0.142221,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e+00
P6016,16273.0,1.072697e+00,0.330463,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.000000e+00
...,...,...,...,...,...,...,...,...
P4030S4,16273.0,1.005776e+00,0.075785,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
P4030S4A1,16179.0,2.871933e+00,0.713877,1.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,9.000000e+00
P4030S5,16273.0,1.028821e+00,0.167308,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
P4040,15804.0,1.255315e+00,0.436052,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00


In [18]:
del area_carac_housing
process_data(area_housing,area_carac, 'area') #base enrichie area, tout pareil 

Nombre obs total carac area
26390
Nombre obs total housing area
8491
Première vérif d'attrion : Pas d'attrition. Voici des stat. desc. de la base enrichie area_carac_housing:


,count,mean,std,min,25%,50%,75%,max
DIRECTORIO,8491.0,6.048616e+06,8018.406511,6.037206e+06,6.041710e+06,6.047906e+06,6.055564e+06,6.062995e+06
SECUENCIA_P,8491.0,1.016724e+00,0.161568,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e+00
ORDEN,8491.0,1.000000e+00,0.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
HOGAR,8491.0,1.016724e+00,0.161568,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e+00
P6016,8491.0,1.084089e+00,0.360598,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.000000e+00
...,...,...,...,...,...,...,...,...
P4030S4,8491.0,1.004475e+00,0.066752,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
P4030S4A1,8453.0,2.872116e+00,0.660454,1.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,9.000000e+00
P4030S5,8491.0,1.004593e+00,0.067621,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
P4040,8452.0,1.143043e+00,0.350137,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00


In [19]:
process_data(r_housing,r_carac, 'rural') #base enrichie rural 

Nombre obs total carac rural
5790
Nombre obs total housing rural
1866
Première vérif d'attrion : Pas d'attrition. Voici des stat. desc. de la base enrichie rural_carac_housing:


,count,mean,std,min,25%,50%,75%,max
DIRECTORIO,1866.0,6.051895e+06,5335.684811,6.036921e+06,6.047277e+06,6.051614e+06,6.054280e+06,6.062951e+06
SECUENCIA_P,1866.0,1.003215e+00,0.065416,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00
ORDEN,1866.0,1.000000e+00,0.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
HOGAR,1866.0,1.003215e+00,0.065416,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00
P6016,1866.0,1.053055e+00,0.301691,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,7.000000e+00
...,...,...,...,...,...,...,...,...
P4030S4,1866.0,1.735263e+00,0.441312,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00
P4030S4A1,494.0,2.271255e+00,1.997794,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,8.000000e+00
P4030S5,1866.0,1.528939e+00,0.499296,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00
P4040,879.0,1.312856e+00,0.463920,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00


In [61]:
total_dataset = pd.concat([area_carac_housing, cabecera_carac_housing, rural_carac_housing], ignore_index=True)

total_dataset.describe().T
total_dataset

,DIRECTORIO,SECUENCIA_P,ORDEN,HOGAR,REGIS,P6016,P6020,P6030S1,P6030S3,P6040,P6050,P6083,P6083S1,P6081,P6081S1,P6080,P6080S1,P6070,P6071,P6071S1,P3147S1,P3147S2,P3147S3,P3147S4,P3147S5,P3147S6,P3147S7,P3147S8,P3147S9,P3147S10,P3147S10A1,P3147S11,P3246,P6090,P6140,P6150,P6100,P6110,P6120,P6125,P6160,P6170,P6175,P6210,P6210S1,P6220,P6269,AREA,ESC,MES,DPTO,fex_c_2011,merge_id,taille_menage,uc,DIRECTORIO_y,SECUENCIA_P_y,P5000,P5010,P5020,P5030,P5040,P5050,P5070,P5080,P5090,P5090S1,P5100,P5110,P5130,P5140,P5210S1,P5210S2,P5210S3,P5210S4,P5210S5,P5210S6,P5210S7,P5210S8,P5210S9,P5210S10,P5210S11,P5210S14,P5210S15,P5210S16,P5210S17,P5210S18,P5210S19,P5210S20,P5210S21,P5210S22,P5210S24,P5220,P5220S1,P6008,P6007,P6007S1,HOGAR_y,P4000,P4010,P4020,P4030S1,P4030S1A1,P4030S2,P4030S3,P4030S4,P4030S4A1,P4030S5,P4040,REGIS_y,AREA_y,MES_y,DPTO_y,fex_c_2011_y,couche_territoriale,CLASE,CLASE_y
0,6037206.0,1.0,1.0,1.0,10,1.0,1.0,1.0,1963.0,58.0,1.0,3.0,NaN,3.0,NaN,6.0,,3.0,1.0,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,,NaN,1.0,1.0,NaN,NaN,1.0,1.0,36341.0,1.0,1.0,2.0,NaN,5.0,11.0,2.0,NaN,05,11.0,12,05,1296.092057,6037206.01.0,2,1.414214,6037206.0,1.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,,NaN,NaN,NaN,800000.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,NaN,1.0,2.0,1.0,4.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,01,05,12,05,1296.092057,area,NaN,NaN
1,6037207.0,1.0,1.0,1.0,10,1.0,1.0,1.0,1979.0,42.0,1.0,2.0,NaN,2.0,NaN,6.0,,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,,NaN,2.0,1.0,NaN,NaN,1.0,1.0,36341.0,2.0,1.0,2.0,NaN,5.0,11.0,2.0,NaN,05,11.0,12,05,1451.291178,6037207.01.0,7,2.645751,6037207.0,1.0,6.0,4.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,,NaN,NaN,NaN,750000.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,4.0,7.0,2.0,NaN,1.0,2.0,1.0,4.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,01,05,12,05,1451.291178,area,NaN,NaN
2,6037209.0,1.0,1.0,1.0,10,1.0,2.0,11.0,1977.0,44.0,1.0,3.0,NaN,3.0,NaN,6.0,,6.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,,NaN,2.0,1.0,NaN,NaN,1.0,1.0,36341.0,2.0,1.0,2.0,NaN,5.0,11.0,2.0,NaN,05,11.0,12,05,1193.103490,6037209.01.0,2,1.414214,6037209.0,1.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,,NaN,NaN,NaN,700000.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,NaN,1.0,2.0,1.0,4.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,01,05,12,05,1193.103490,area,NaN,NaN
3,6037210.0,1.0,1.0,1.0,10,1.0,1.0,8.0,1972.0,49.0,1.0,2.0,NaN,3.0,NaN,6.0,,2.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,,NaN,2.0,1.0,NaN,NaN,1.0,1.0,36341.0,2.0,1.0,2.0,NaN,5.0,11.0,2.0,NaN,05,11.0,12,05,1155.402197,6037210.01.0,3,1.732051,6037210.0,1.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,,NaN,NaN,NaN,750000.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,3.0,2.0,NaN,1.0,2.0,1.0,4.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,01,05,12,05,1155.402197,area,NaN,NaN
4,6037211.0,1.0,1.0,1.0,10,1.0,2.0,8.0,1986.0,35.0,1.0,2.0,NaN,2.0,NaN,6.0,,4.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,1.0,1.0,NaN,NaN,1.0,5.0,NaN,2.0,1.0,2.0,NaN,6.0,5.0,4.0,NaN,05,16.0,12,05,1094.942950,6037211.01.0,3,1.732051,6037211.0,1.0,4.0,3.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,,NaN,NaN,NaN,740000.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,3.0,2.0,NaN,1.0,2.0,1.0,4.0,1.0,3.0,1.0,1.0,1.0,3.0,1.0,1.0,01,05,12,05,1094.942950,area,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26625,6062932.0,1.0,1.0,1.0,10,1.0,2.0,6.0,1965.0,56.0,1.0,3.0,NaN,3.0,NaN,6.0,,4.0,NaN,NaN,NaN,NaN,N

In [21]:
#documenter rapidement pourquoi il y a plus de colonnes dans la base cabecera que rural et area

var_area = area_carac_housing.columns.tolist()
var_rural = rural_carac_housing.columns.tolist()
var_caba = cabecera_carac_housing.columns.tolist()

not_common_var_caba_area = set(var_caba) - set(var_area) #caba possede CLASE : rural ou pas rural
not_common_var_caba_rural = set(var_caba) - set(var_rural) #caba possede AREA : meme info que DPTO
not_common_var_area_rural_clase =  set(var_rural) - set(var_area) #rural possede CLASE
not_common_var_area_rural_AREA =  set(var_area) - set(var_rural) #area possede AREA
#donc normal de se retrouver avec 117 colonnes

not_common_var_caba_area 
not_common_var_caba_rural
not_common_var_area_rural_clase
not_common_var_area_rural_AREA
len(var_rural)
len(var_area)
len(var_caba)

rural_area_var = set(var_area) & set(var_rural)
len(rural_area_var)

113

Ci- dessous, on termine le preprocessing de la base, avant traitement des variables. Il nous reste à vérifier qu'il n'y a pas de doublons dans notre base finale total_dataset. On decouvre finalement que la base cabecera_carac_housing inclut les donnees de la base area_carac_housing. Cependant, Il reste tout de meme utile d'avoir append (area+cabecera+housing) au préalable via la base area (au lieu d'utiliser directement la base cabecera): sans cela, on n'aurait pas pu avoir l'information de la couche territoriale ('couche_territoriale'). On verra effectivement que l'information dans 'CLASE' ne permet pas de retrouver les memes nombres d'individus dans les 3 couches territoriales retrouvable via la procedure append. 

En somme, on aurait pu supprimer les duplicatas dans total_dataset en gardant un unique de merge_id depuis la base cabecera_carac_housing puis prendre l'information de 'CLASE' et définir le centre urbain comme 'non rural' et le périurbain comme 'rural' (et ainsi construire notre variable territoire en fonction de cette information dans CLASE). Mais avec cette classification, on perd de la finesse entre l'ultra centre urbain (area) et le périurbain de ce centre urbain (cabecera), qui justement illustre l'hypothèse monocentrique de l'arbitrage logement-transport (donc centre_urbain vs. périurbain > urbain vs. rural). 

Premieres limites observées de cette base de donnée du DANE (pour une analyse monocentrique des postes de dépenses) : la granularité territoriale des données. En effet, il n'y a pas de variable qui permet de connaitre la commune de provenance de l'individu. L'echelle communale permettrait d'isoler cet hypercentre de sa zone periurbaine et ainsi, d'avoir une classification de territoire plus fine. 
- on contourne le problème de manque de données communales avec la variable couche_territoriale. Ainsi, on va construire notre variable territoire 'territory' (voir tableau dans 0_main ; 2_data_cleaning pour le traitement) avec l'hypercentre qui est la couche_territoriale 'area' et le reste du departement correspondra au périurbain (couche_territoriale cabecera ou rural).

Toutefois, on observe un autre problème dans la base : pour le département de Bogota, la base cabecera_carac_housing n'est pas populée (contrairement aux autres grandes villes dans la base), ce qui signifie qu'on ne peut pas differencier les individus vivant dans le centre du departement de Bogota et ceux en périphérie. Ici, le manque de granularité empêche une analyse monocentrique des dépenses intra-territoriale plus fine pour Bogota et nous oblige à procéder différemment pour étudier le pluricentrisme à Bogota 
- on utilisera une spécificité territoriale en Colombie, le rang d'imposition sur le logement, la variable Utilrank

In [25]:
t = total_dataset[total_dataset['couche_territoriale']=='area'] #on veut prendre seulement les ruraux dans les 8491 // ou du moins, savoir si on peut categoriser 

In [38]:
t = total_dataset[total_dataset['couche_territoriale'] == 'area']['merge_id'].tolist()

result_subset = cabecera_carac_housing[cabecera_carac_housing['merge_id'].isin(t)]

In [41]:
result_subset['CLASE'].value_counts()

CLASE
1    8491
Name: count, dtype: int64

In [42]:
filtered_subset = cabecera_carac_housing[~((cabecera_carac_housing['merge_id'].isin(t)) & (cabecera_carac_housing['couche_territoriale'] == 'cabecera'))]


In [43]:
filtered_subset.count() #cabecera sans les area. ca revient au meme de garder la base cabecera, c'est juste qu'on peut label les area lors du append

DIRECTORIO             7782
SECUENCIA_P            7782
ORDEN                  7782
HOGAR                  7782
REGIS                  7782
                       ... 
CLASE_y                7782
MES_y                  7782
DPTO_y                 7782
fex_c_2011_y           7782
couche_territoriale    7782
Length: 117, dtype: int64

In [65]:
total_dataset_new = pd.concat([area_carac_housing, filtered_subset, rural_carac_housing], ignore_index=True)


In [88]:
total_dataset_new[total_dataset_new['DPTO']=='11'].count() #723 de bogota  
total_dataset_new[(total_dataset_new['DPTO']=='11') & (total_dataset_new['couche_territoriale']=='cabecera')].count() #0 ! 
total_dataset_new[(total_dataset_new['DPTO']=='11') & (total_dataset_new['couche_territoriale']=='area')].count() #723. popule totalement, sans cabecera ni rural 


#on prend l'exemple d'un autre grand departement : valle del cauca (centre urbain: Cali)
total_dataset_new[total_dataset_new['DPTO']=='76'].count()#988 au total dans la region
total_dataset_new[(total_dataset_new['DPTO']=='76') & (total_dataset_new['couche_territoriale']=='cabecera')].count() #241
total_dataset_new[(total_dataset_new['DPTO']=='76') & (total_dataset_new['couche_territoriale']=='area')].count() #697
total_dataset_new[(total_dataset_new['DPTO']=='76') & (total_dataset_new['couche_territoriale']=='rural')].count() #50

# on prend par exemple l'aire urbaine de Antioquia ( Medellin comme area centre) 
total_dataset_new[total_dataset_new['DPTO']=='05'].count()#1235 au total dans la region
total_dataset_new[(total_dataset_new['DPTO']=='05') & (total_dataset_new['couche_territoriale']=='cabecera')].count() #103
total_dataset_new[(total_dataset_new['DPTO']=='05') & (total_dataset_new['couche_territoriale']=='area')].count() #1000
total_dataset_new[(total_dataset_new['DPTO']=='05') & (total_dataset_new['couche_territoriale']=='rural')].count() #103

#regress avec utilrank pour seulement bogota

DIRECTORIO             132
SECUENCIA_P            132
ORDEN                  132
HOGAR                  132
REGIS                  132
                      ... 
DPTO_y                 132
fex_c_2011_y           132
couche_territoriale    132
CLASE                  132
CLASE_y                132
Length: 117, dtype: int64

In [86]:
total_dataset_new[total_dataset_new['DPTO']=='76']['CLASE'].value_counts()#on aurait effectivement eu la mauvaise information, pas en couche territoriale comme on le veut
# on n'aurait pas su quoi faire des NaN

CLASE
1    241
2     50
Name: count, dtype: int64

In [91]:
#enfin, on verifie qu'on n'a plus de duplicata apres ce traitement 
total_dataset['merge_id'].nunique() #18139, avec 26630 count() on se souvient, dans total_dataset
total_dataset_new['merge_id'].count() #18139
total_dataset_new['merge_id'].nunique() #18139, tout est bon donc


18139

    La méthode d'echantillonage des bases du GEIH2021 

Pour s'assurer que étape de préparation de la base enrichie soit correct, on peut vérifier que l'estimation du total sur le champ étudié corresponde aux indicateurs nationaux. Il faut que notre échantillon (base finale, "total_dataset") soit représentative de la population du champ étudié.

Le champ de l'échantillonage de la DANE se fait sur la population active, càd qui fait partie de la workforce (voir [DANE (2023)](https://microdatos.dane.gov.co/index.php/catalog/701/study-description#metadata-sampling) : "Weighting
Se diseñó una muestra para obtener una precisión esperada medida en términos del error de muestreo menor o igual a 5% con un nivel de confiabilidad del 95%, para los principales indicadores de fuerza laboral nacionales.")

On connait les données annuelles de la population formant la main d'oeuvre (workforce, active population) en Colombie ([DANE (2021)](https://www.dane.gov.co/files/investigaciones/boletines/ech/ech/CP_empleo_dic_21.pdf]),[World Bank (2023)](https://data.worldbank.org/indicator/SL.TLF.TOTL.IN?locations=CO)) et en trimestre ([OECD(2023)](https://stats.oecd.org/), aller à Labour > Short-Term Statistics > Short-Term Labour Market Statistics > Active Population > Q4 2021, Colombia), d'environ 23 millions de personnes au Q42021 et 2021 annuellement. Donc en sommant les probabilités d'inclusion de notre base finale, on devrait retrouver le total de la population active, c'est qu'on est en mesure de retrouver :

(Note : fex_c_2011 : c'est le poids de sous echantillonage de l'individu (qui est donc representatif de la population totale)
Voir la documentation téléchargeable [ici](https://microdatos.dane.gov.co/index.php/catalog/701/pdf-documentation)

In [92]:
pd.crosstab(total_dataset_new['fex_c_2011'] == total_dataset_new['fex_c_2011_y'],columns="count") #memes variables, on peut utiliser l'un ou l'autre interchangeablement, pour les poids d'echantillonage
# toute la base popule fex_c_2011

col_0,count
row_0,
True,18139


In [93]:
total_dataset_new['fex_c_2011'].sum() #on trouve les memes ordres de grandeurs, donc le preprocessing semble correct. Ne pas oublier que notre base est mensuelle et que le workforce est saisonnier et varie sur l'annee entiere : les individus entrent et sortent du marche du travail selon les mois de l'annee, et ainsi, une base de donnee sur le champ de la population active n'est pas cylindree sur toute l'annee. et le total sur l'annee de la population active est aussi superieure, ce n'est pas une moyenne du workforce par mois

15817802.45622806

In [98]:
#1. on prend deux autres mois au hasard, pour voir que les ordres de grandeurs sont bien respectees 
    # enero, janvier 2021. cette base a ete cleanée par la DANE en 2023, sans les 3 couches territoriales, donc en verifiant avec elle, on est surs de pas se tromper
        # on verra dans 1.extract qu'il nous est essentiel d'avoir cette stratification en 3 couches territoriales, par manque de granularité des donnees (donnees departementales et non communales) 
    # julio, juillet 2021. en été, on va voir qu'il y a clairement moins d'individus 

#2. au passage, on va aussi expliquer pourquoi on ne decide pas d'enrichir notre base avec plusieurs mois
    # l'avantage d'etendre la base à l'année serait: (1) soit d'augmenter le nombre de points de données, (2) soit d'avoir les dépenses annuelles des individus
    # (2) on ne peut tout simplement pas 
        # car la 'base annuelle' (append de toutes les bases area+cabecera+rual de chaque mois) n'est pas cylindree - normal etant donne la complexite de garder des individus dans une enquete sur une longue duree. 
        # on voit rapidement que les individus de mois en mois ne sont pas les memes, donc à part augmenter le nombre d'observations, cela n'ajoute rien à l'étude
    # (1) ajouter des mois supplémentaires élargit certes l'échantillon
        # mais cela signifie aussi qu'il sera plus compliqué de controler et comprendre d'ou viennent les problemes rencontrés
        # il faudrait ajouter des effets fixes notamment dues a la saisonnalité des donnees, par exemple du travail (et donc des salaires, et potentiellement du loyer)
        # donc plus judicieux de travailler sur une taille reduite d'echantillon pour le cadre de notre etude
    # au bout du compte, on utilise des donnees mensuelles, donc on prend juste un mois arbitrairement, qui ne soit pas trop sujet à la saisonalité, et qui nous permet d'avoir 3 couches territoriales (Diciembre.dta)

# enero 
zip_file_path = 'data/Enero.dta.zip'
output_directory = '/home/onyxia/work/extracted_files' #ici, on met les donnees dans le serveur du SSP lab, pour eviter de faire des commits directs sur le clone github: en effet, si on ajoute toutes nos donnees unziped sur le dossier a commit, on va creer des erreurs
download_and_extract_zip_from_github(github_owner, github_repo, zip_file_path, output_directory)


Files extracted to '/home/onyxia/work/extracted_files'


In [99]:
# julio 
zip_file_path = 'data/Julio.dta.zip'
output_directory = '/home/onyxia/work/extracted_files' #ici, on met les donnees dans le serveur du SSP lab, pour eviter de faire des commits directs sur le clone github: en effet, si on ajoute toutes nos donnees unziped sur le dossier a commit, on va creer des erreurs
download_and_extract_zip_from_github(github_owner, github_repo, zip_file_path, output_directory)

Files extracted to '/home/onyxia/work/extracted_files'


In [117]:
#1. Les DIRECTORIO dans les bases Enero et Julio ne sont pas inclus dans les DIRECTORIO de notre base 
# donc deja rien que pour les ID des individus, on ne peut pas identifier ceux de Diciembre.dta avec les bases Enero.dta et Julio.dta

#enero, toutes couches territoriales
enero_carac_all = pd.read_stata("/home/onyxia/work/extracted_files/Enero.dta/Características generales, seguridad social en salud y educación.DTA")
enero_carac_all['DIRECTORIO'].min() #1000950.0
enero_carac_all['DIRECTORIO'].max() #4025401.0 

#julio
!pip install pyreadstat
import pyreadstat    #ancienne stata dta, version 110
# area
data, meta  = pyreadstat.read_dta("/home/onyxia/work/extracted_files/Julio.dta/╡rea - Características generales (Personas).dta")
julio_carac_area = data
julio_carac_area['DIRECTORIO'].min() #5494143
julio_carac_area['DIRECTORIO'].max() # 5518734
#cabecera
data, meta  = pyreadstat.read_dta("/home/onyxia/work/extracted_files/Julio.dta/Cabecera - Características generales (Personas).dta")
julio_carac_cabecera = data 
julio_carac_cabecera['DIRECTORIO'].min() #5494060
julio_carac_cabecera['DIRECTORIO'].max() # 5518734
#resto 
data, meta  = pyreadstat.read_dta("/home/onyxia/work/extracted_files/Julio.dta/Resto - Características generales (Personas).dta")
julio_carac_rest = data
julio_carac_rest['DIRECTORIO'].min() #5498841
julio_carac_rest['DIRECTORIO'].max() #5518712


In [132]:
#2. on verifie l'ordre de grandeur de fex_c_2011

# maintenant, on prend les dataset sur le logement

#enero
# il n'y aura pas de duplicatas, vu que la base est cleanée 
enero_housing_all = pd.read_stata("/home/onyxia/work/extracted_files/Enero.dta/Datos del hogar y la vivienda.DTA")
enero_housing_all['FEX_C18'].sum() #16315983.106865954, 16 millions on est bon

#julio
data, meta  = pyreadstat.read_dta("/home/onyxia/work/extracted_files/Julio.dta/╡rea - Vivienda y Hogares.dta")
julio_housing_area = data
data, meta  = pyreadstat.read_dta("/home/onyxia/work/extracted_files/Julio.dta/Cabecera - Vivienda y Hogares.dta")
julio_housing_cabecera = data
data, meta  = pyreadstat.read_dta("/home/onyxia/work/extracted_files/Julio.dta/Resto - Vivienda y Hogares.dta")
julio_housing_rural = data
#pareil que dans Diciembre.dta, les area sont doublés dans les cabecera 
julio_housing_area['merge_id'] = julio_housing_area['DIRECTORIO'].astype(str) + julio_housing_area['HOGAR'].astype(str)
julio_housing_area['merge_id'].nunique() #8866
julio_housing_cabecera['merge_id'] = julio_housing_cabecera['DIRECTORIO'].astype(str) + julio_housing_cabecera['HOGAR'].astype(str)
julio_housing_cabecera['merge_id'].nunique() #16811
julio_housing_cabecera_area = pd.concat([julio_housing_cabecera, julio_housing_area], ignore_index=True)
julio_housing_cabecera_area['merge_id'].nunique() #16811, on retombe sur nunique de cabecera, donc les ID appendé depuis area sont doublés dans cabecera

julio_housing_cabecera_rural = pd.concat([julio_housing_cabecera, julio_housing_rural], ignore_index=True)
julio_housing_cabecera_rural['fex_c_2011'].sum() #15890691.490340695 c'est ok aussi




/tmp/ipykernel_5445/1444633710.py:7: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  enero_housing_all = pd.read_stata("/home/onyxia/work/extracted_files/Enero.dta/Datos del hogar y la vivienda.DTA")


15890691.490340695

In [136]:
julio_housing_cabecera_area = pd.concat([julio_housing_cabecera, julio_housing_area], ignore_index=True)
julio_housing_cabecera_area['merge_id'].nunique() #16811

16811

    2.3 Export de la base enrichie, non clean

In [95]:
total_dataset_new.to_stata('data_base_enrichie_non_clean.dta',write_index=False)

### Bibliographie de ce Notebook :

    Méthodologie employée pour la composition des ménages (unités de consommation) :

[OECD (2020) Compare your income methodology](https://www.oecd.org/statistics/Compare-your-income-methodology.pdf)

[OECD (n.d) Adjusting household incomes: equivalence scales](https://www.oecd.org/els/soc/OECD-Note-EquivalenceScales.pdf)

    Données sur la population active Colombienne en 2021 : 

[DANE (2021) Comunicado de prensa - Indicadores de mercado laboral](https://www.dane.gov.co/files/investigaciones/boletines/ech/ech/CP_empleo_dic_21.pdf), retrouvable [ici](https://www.dane.gov.co/index.php/estadisticas-por-tema/mercado-laboral/empleo-y-desempleo/mercado-laboral-historicos) 

[OECD(2023) Short-Term Labour Statistics : Active Population ](https://stats.oecd.org/) (aller à Labour > Short-Term Statistics > Short-Term Labour Market Statistics > Active Population > Q4 2021, Colombia)

[World Bank (2023) Labor Force, total - Colombia](https://data.worldbank.org/indicator/SL.TLF.TOTL.IN?locations=CO)
